In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix
from torch import nn
from sklearn.metrics import accuracy_score as acc

# CUDA - CNN with Class

In [2]:
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])
train = datasets.MNIST('./Data', train= True, download= True, transform= transforms)
test = datasets.MNIST('./Data', train= False, download= True, transform=transforms)
train_loader = DataLoader(train, batch_size=64, shuffle= True)
test_loader = DataLoader(test, batch_size= 64, shuffle= True)

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride= 2),
            nn.Conv2d(32, 64, kernel_size=3), 
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2), 
            nn.Conv2d(64, 32, kernel_size= 3),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride= 2)
        )
        self.classify_head = nn.Sequential(
            nn.Linear(32, 20, bias= True),
            nn.Linear(20, 10, bias= True)
        )
    
    def forward(self, x):
        return self.classify_head(self.net(x).reshape(-1, 32))

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
model = CNN().to(device=device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [6]:
epochs = int(input("Enter Number of Epochs : "))
epochs

11

In [7]:
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for input, target in train_loader:
        input, target = input.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        running_loss += loss.item()
        optimizer.step()
    print(f'Epoch - {epoch+1}, loss = {running_loss}')

Epoch - 1, loss = 2158.0305013656616
Epoch - 2, loss = 2143.6634936332703
Epoch - 3, loss = 2125.5129721164703
Epoch - 4, loss = 2093.9018342494965
Epoch - 5, loss = 2033.62961935997
Epoch - 6, loss = 1902.476318359375
Epoch - 7, loss = 1616.9901477098465
Epoch - 8, loss = 1216.7833691835403
Epoch - 9, loss = 878.488124191761
Epoch - 10, loss = 658.9146489799023
Epoch - 11, loss = 518.8318795859814


In [8]:
torch.save(model, './Models/CNN-CUDA.pt')

In [9]:
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for input, target in test_loader:
        output = model(input.to('cuda'))
        val, index = torch.max(output, 1)
        all_preds.extend(index.to('cpu'))
        all_labels.extend(target.to('cpu'))
cm = confusion_matrix(all_labels, all_preds)
print(cm)
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

[[ 939    0   14    0    1    8   12    5    1    0]
 [   0 1097    3    1    0    0    0    5   29    0]
 [   9    8  882   17   12    5   16   42   11   30]
 [   0    2   18  903    0   21    0   14   30   22]
 [   1    0    2    0  898    0   34    0    5   42]
 [  13    3    8   31   13  669   10   15  101   29]
 [  19    5   18    0   17   14  859    0   26    0]
 [   5   11   57   30    2    3    0  898    3   19]
 [   1   15    9   71   21   71   13    3  730   40]
 [   7    1   19   10   31   39    2   12   36  852]]
38150


In [10]:
ac = acc(all_labels, all_preds)
print(f"Accuracy : {ac}")

Accuracy : 0.8727
